## Configuration

In [1]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# imports
import os

import torch, torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import CIFAR10

torch.__version__, torchvision.__version__
torch.cuda.is_available()
# Install Albumentation library
%pip install albumentations -qqq
# Install LR finder
%pip install torchinfo torch_lr_finder -qqq
# Install GradCam
%pip install grad-cam -qqq

%pip install tqdm -qqq

%pip install torchscan --no-deps -qqq

%pip install seaborn -qqq

%pip install gradio -qqq

%pip install pytorch-lightning -qqq

%pip install torchmetrics -qqq

%pip install wandb -qqq

os.environ['KMP_DUPLICATE_LIB_OK']='True'
import seaborn as sn  # for heatmaps
import math
from collections import OrderedDict
import sys
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR, OneCycleLR, ReduceLROnPlateau
import torchmetrics
from torchmetrics import Accuracy
import matplotlib.pyplot as plt

from torch_lr_finder import LRFinder
import copy
from torchvision.utils import make_grid, save_image

import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelSummary

# Weights & Biases
import wandb
from pytorch_lightning.loggers import WandbLogger

import gradio as gr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!git clone https://github.com/nanekja/pytorch_utils -qqq

sys.path.append('./pytorch_utils/')
import main
import utils
import transform
import train1
import test1
import dataloader
import albumentation
import gradcam

sys.path.append('./pytorch_utils/models/')
import c_resnet

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

fatal: destination path 'pytorch_utils' already exists and is not an empty directory.


/home/nanekja/.local/lib/python3.10/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


ModuleNotFoundError: No module named 'gradcam'

## DATA

In [ ]:
batch_size = 512
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class CIFAR10DataModule(LightningDataModule):
    def __init__(self, batch_size=batch_size, num_workers=4):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        datasets.CIFAR10(root='./data', train=True, download=True)
        datasets.CIFAR10(root='./data', train=False, download=True)

    def setup(self, stage=None):

        transform_train = A.Compose([A.Normalize(mean=(0.49139968, 0.48215841, 0.44653091), std=(0.24703223, 0.24348513, 0.26158784),),
        A.PadIfNeeded(40,40),
        A.RandomCrop(32,32),
        A.HorizontalFlip(),
        A.Cutout(num_holes=1, max_h_size=16, max_w_size=16, fill_value=[0.49139968, 0.48215841, 0.44653091], always_apply=True, p=0.50)])

        transform_test = A.Compose([A.Normalize(mean=[0.49139968, 0.48215841, 0.44653091], std=[0.24703223, 0.24348513, 0.26158784],)])

        if stage == 'fit' or stage is None:
            self.trainset = datasets.CIFAR10(root='./data', train=True, transform=transform_train)
            self.testset = datasets.CIFAR10(root='./data', train=False, transform=transform_test)

    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.testset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.testset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)

In [ ]:
class Lit_c_resnet(LightningModule):

    def __init__(self, n_classes=10, lr=1e-3):
        super().__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, 3, stride=1, padding=1, bias=False), nn.BatchNorm2d(64), nn.ReLU())
        self.layer2 = nn.Sequential(nn.Conv2d(64, 128, 3, stride=1, padding=1, bias=False), nn.MaxPool2d(2, 2), nn.BatchNorm2d(128), nn.ReLU())

        self.resblock1 = self.resblock(128, 128, 3)

        self.layer3 = nn.Sequential(nn.Conv2d(128, 256, 3, stride=1, padding=1, bias=False), nn.MaxPool2d(2, 2), nn.BatchNorm2d(256), nn.ReLU())
        self.layer4 = nn.Sequential(nn.Conv2d(256, 512, 3, stride=1, padding=1, bias=False), nn.MaxPool2d(2, 2), nn.BatchNorm2d(512), nn.ReLU())

        self.resblock2 = self.resblock(512, 512, 3)

        self.pool = nn.MaxPool2d(4,4)
        
        self.fc = nn.Linear(in_features=512,out_features=n_classes, bias = False)

        self.accuracy = Accuracy()

    def resblock(self, in_channels, out_channels, kernel_size):
        conv = nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
        )
        return conv


    def forward(self, x):
        x = self.layer1(x) ## Input size = 32x32, output size = 32x32
        x = self.layer2(x) ## Input size = 32x32, output size = 16x16
        
        res_1 = self.resblock1(x) ## Input size = 16x16, output size = 16x16
        x = x + res_1
        
        x = self.layer3(x) ## Input size = 16x16, output size = 8x8
        x = self.layer4(x) ## Input size = 8x8, output size = 4x4
        
        res_2 = self.resblock2(x) ## Input size = 4x4, output size = 4x4
        x = x + res_2 
        
        x = self.pool(x) ## Input size = 4x4, output size = 1x1
        x = x.view(x.size(0), -1)
        
        x = self.fc(x)
        x = x.view(-1, 10)
        
        return F.softmax(x, dim=-1)


    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        #loss = self.criterion(logits, y)
        #preds = torch.argmax(logits, dim=1)
        #self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", torchmetrics.functional.accuracy(logits, y), prog_bar=True)
        #return loss
        

        # Log validation loss (will be automatically averaged over an epoch)
        #self.log('valid_loss', loss)

        # Log metrics
        #self.log('valid_acc', torchmetrics.functional.accuracy(logits, y))

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)
        
    
    def configure_optimizers(self):
        '''defines model optimizer'''
        return optim.SGD(self.parameters(), lr=self.lr, momentum=0.9)



In [ ]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
AVAIL_GPUS

In [6]:
model = Lit_c_resnet()
trainer = Trainer(callbacks=[ModelSummary(max_depth=-1)], precision = 16, gpus = AVAIL_GPUS, max_epochs = 5, progress_bar_refresh_rate=10)
trainer.fit(model)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainer.test()

In [ ]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [ ]:
torch.save(model.state_dict(), "model.pth")

